In [5]:
import torch

from pina.problem import SpatialProblem, TimeDependentProblem
from pina.operators import nabla, grad, div, curl, advection
from pina import Condition, Span, LabelTensor


class QGE(SpatialProblem, TimeDependentProblem):

    output_variables = ['q', 'si']
    spatial_domain = Span({'x': [0, 1], 'y': [-1, 1]})
    temporal_domain = Span({'t': [0, 1]})
    
    def rand_choice_integer_Data(self):
        pass 
    
    def eq1(input_, output_):
        nu = 0.0022
        Re = 100 / nu
        
        #convective term
        si_curl = curl(output_.extract(['si']), input_, d = ['x', 'y'])
        convective_ = advection(output_.extract(['q']), input_, velocity_field = si_curl, d = ['x', 'y'])
        
        #diffusive term
        diffusive_ = nabla(output_.extract(['q']), input_, d = ['x', 'y'])   
        
        #transient term 
        du = grad(output_.extract(['q']), input_)
        transient_ = du.extract('dqdt')
        
        #forcing term
        force_ = torch.sin(torch.pi * input_.extract(['y']))
        
        return transient_ + convective_ - (1/Re) * diffusive_ - force_
    
    def eq2(input_, output_):
        r0 = 0.0036
        
        #second equation
        output = output_.extract(['q']) + r0 * nabla(output_.extract(['si']), input_, d = ['x', 'y']) - input_.extract(['y'])
        return output
        
    def continuity(input_, output_):
        si_curl = curl(output_.extract(['si']), input_, d = ['x', 'y'])
        return div(si_curl, input_, d = ['x', 'y'])

    def initial(input_, output_):
        value = 0.0
        return output_.extract(['si']) - value
    
    def zeta(input_, output_):
        value = input_.extract(['y'])
        return output_.extract(['q']) - value
    
    def si(input_, output_):
        si_expected = 0.0
        return output_.extract(['si']) - si_expected
        
    
    conditions = {
        't0': Condition(Span({'x': [0, 1], 'y': [-1, 1], 't' : 0}), initial),
        
        'upper': Condition(Span({'x':  [0,1], 'y': 1, 't': [0,100]}), [si, zeta]),
        'fixedWall1': Condition(Span({'x':  0, 'y': [-1,1], 't': [0,100]}), [si, zeta]),
        'fixedWall2': Condition(Span({'x':  1, 'y': [-1,1], 't': [0,100]}), [si, zeta]),
        'fixedWall3': Condition(Span({'x':  [0,1], 'y': -1, 't': [0,100]}), [si, zeta]),
        
        'D': Condition(Span({'x': [0, 1], 'y': [-1, 1], 't': [0, 100]}), [eq1, eq2]),
    }


In [6]:

from torch.nn import Softplus
from pina import PINN, LabelTensor, Plotter
from pina.model import FeedForward

#args
id_run = 0
save = True

qge_problem = QGE()
model = FeedForward(
    layers=[10, 10, 10, 10],
    output_variables=qge_problem.output_variables,
    input_variables=qge_problem.input_variables,
    func=Softplus,
)
pinn = PINN(
    qge_problem,
    model,
    lr=0.01,
    error_norm='mse',
    regularizer=1e-8)

if save:
    pinn.span_pts(
            {'n': 25, 'mode': 'grid', 'variables': 't'},
            {'n': 4, 'mode': 'grid', 'variables': 'x'},
            {'n': 8, 'mode': 'grid', 'variables': 'y'},
            locations=['t0', 'upper','fixedWall1','fixedWall2','fixedWall3', 'D'])
    
    pinn.train(15000, 1500)
    with open('qge_history_{}.txt'.format(id_run), 'w') as file_:
        for i, losses in pinn.history_loss.items():
            file_.write('{} {}\n'.format(i, sum(losses)))
    pinn.save_state('pina.qge')
else:
    pinn.load_state('pina.qge')
    plotter = Plotter()
    plotter.plot(pinn, components='si')
    plotter.plot_loss(pinn)


              sum          t0initial    uppersi      upperzeta    fixedWall1si fixedWall1ze fixedWall2si fixedWall2ze fixedWall3si fixedWall3ze Deq1         Deq2         
[epoch 00000] 5.737945e+00 4.274926e-02 4.757062e-01 1.147866e+00 4.670334e-01 4.399314e-01 4.796188e-01 4.410101e-01 4.707945e-01 8.952271e-01 4.375463e-01 4.404623e-01 
              sum          t0initial    uppersi      upperzeta    fixedWall1si fixedWall1ze fixedWall2si fixedWall2ze fixedWall3si fixedWall3ze Deq1         Deq2         
[epoch 00001] 4.111963e+00 9.399164e-02 6.102233e-02 9.515398e-01 5.789971e-02 4.373238e-01 6.041072e-02 4.368309e-01 5.727151e-02 1.081058e+00 4.375400e-01 4.370746e-01 
              sum          t0initial    uppersi      upperzeta    fixedWall1si fixedWall1ze fixedWall2si fixedWall2ze fixedWall3si fixedWall3ze Deq1         Deq2         
[epoch 01500] 1.394918e-01 6.277619e-03 4.487215e-04 7.418900e-04 3.138260e-02 2.786314e-03 3.203063e-02 2.293385e-03 2.459138e-04 6.479413e-04 5